In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
#read the input
carDf= pd.read_csv('newCardata.csv')
carDate= pd.read_csv('dataWithDate.csv')
carLable= carDate['FraudFound']

#drop the Accident Date and Claim Date
carDate.drop(['FraudFound'],inplace=True,axis=1)
carDate.drop(['AccidentDate'],inplace=True,axis=1)
carDate.drop(['ClaimDate'],inplace=True,axis=1)

carDate.head()

print(type(carLable[0]))
print(carLable.shape)

<class 'numpy.int64'>
(15419,)


In [4]:

#select all the attributes of type object
car_object= carDate.select_dtypes(include=['object']).copy()

#drop the attributes of type object
carDate.drop(carDate.select_dtypes(['object']),inplace=True,axis=1)

carDate.head()
#print(type(carDate))

,RepNumber,Deductible,DriverRating,DayDiff
0,12,300,1,0
1,15,400,4,5
2,7,400,3,13
3,4,400,2,20
4,3,400,1,15


In [5]:
#one-hot representation preprocessing

car_copy = car_object.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['PolicyType'],prefix=['PolicyType'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfPolicyHolder'],prefix=['AgeOfPolicyHolder'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])

car_copy.head()
print(type(car_copy))

<class 'pandas.core.frame.DataFrame'>


In [6]:
car_copy['RepNumber']= carDate['RepNumber']
car_copy['Deductible']= carDate['Deductible']
car_copy['DriverRating']= carDate['DriverRating']
car_copy['DayDiff']= carDate['DayDiff']

car_copy.head(3)

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNumber,Deductible,DriverRating,DayDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,12,300,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,15,400,4,5
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,7,400,3,13


In [7]:
#normalization of feature to bring the value in the range [0,1]
minMaxScale = MinMaxScaler()
carDateNormalized = minMaxScale.fit_transform(car_copy)
#print(car_df_Scale)

#converting numpyarry to dataframe
carDateNormalized = pd.DataFrame(carDateNormalized)
carDateNormalized.head(3)
print(type(carDateNormalized))

<class 'pandas.core.frame.DataFrame'>


/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [8]:
carArray= carDateNormalized.values
labels= carLable.values
print(type(carArray))

<class 'numpy.ndarray'>


In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(784, 32)

    def forward(self, x):
        return F.sigmoid(self.fc1(x))

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(32, 784)
        
    def forward(self, x):
        return F.sigmoid(self.fc1(x))

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, x):
        return self.fc2(self.fc1(x))

model = AutoEncoder()
optimizer = optim.Adam(model.parameters(), lr=0.5)
'''
for epoch in range(1, 201):
    train(epoch)
    test(epoch, validation)
    '''

'\nfor epoch in range(1, 201):\n    train(epoch)\n    test(epoch, validation)\n    '